In [ ]:
# %%
import sys
import os
import luigi
import sciluigi as sl
import logging

sys.path.append('..')
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('SMB').setLevel(logging.WARNING)
logging.getLogger('napari').setLevel(logging.WARNING)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('in_n_out').setLevel(logging.WARNING)
logging.getLogger('numcodecs').setLevel(logging.WARNING)
logging.getLogger('numba').setLevel(logging.WARNING)
logging.getLogger('luigi').setLevel(logging.WARNING)
logging.getLogger('numexpr').setLevel(logging.WARNING)
logging.getLogger('luigi-interface').setLevel(logging.WARNING)
logging.getLogger('sciluigi-interface').setLevel(logging.WARNING)
logging.getLogger('cellpose').setLevel(logging.WARNING)

from src import Receipt
from src.steps import get_task

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


In [2]:
class AngelFISHLuigiTask(sl.ExternalTask):
    receipt_path = luigi.Parameter()
    step_name = luigi.Parameter()
    output_path = luigi.Parameter()

    @property
    def task_name(self):
        return self.step_name

    def out_doneflag(self):
        return sl.TargetInfo(self, self.output_path)

    def run(self):
        receipt = Receipt(path=self.receipt_path)
        task_class = get_task(receipt['steps'][self.step_name]['task_name'])
        task = task_class(receipt, self.step_name)
        task.process()  # this creates the .txt file

In [ ]:
class AngelFISHWorkflow(sl.WorkflowTask):
    receipt_path = luigi.Parameter()

    def out_doneflag(self):
        receipt = Receipt(path=self.receipt_path)
        return sl.TargetInfo(self, f'{receipt['meta_arguments']['analysis_name']}.txt')

    def workflow(self):
        receipt = Receipt(path=self.receipt_path)
        step_order = receipt['step_order']

        previous_task = None
        task_refs = []
        for step_name in step_order:
            path = os.path.join(receipt['dirs']['status_dir'], f'step_{step_name}.txt')
            step_task = self.new_task(
                step_name,
                AngelFISHLuigiTask,
                receipt_path=self.receipt_path,
                step_name=step_name,
                output_path=path
            )

            # Add dependency chain
            if previous_task is not None:
                step_task.in_upstream = previous_task.out_doneflag

            previous_task = step_task
            task_refs.append(step_task)
            
        return task_refs

In [4]:
luigi.build(AngelFISHWorkflow(receipt_path=r'C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json').workflow(), local_scheduler=True)

INFO:luigi:logging configured by default settings
DEBUG: Checking if AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name=download_data, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json, step_name=download_data, output_path=c:\Users\formanj\GitHub\AngelFISH\database\SS004_10min_100nM_7\default_name\status\step_download_data.txt) is complete
DEBUG:luigi-interface:Checking if AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name=download_data, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json, step_name=download_data, output_path=c:\Users\formanj\GitHub\AngelFISH\database\SS004_10min_100nM_7\default_name\status\step_download_data.txt) is complete
2025-07-30 13:44:23 |     INFO | Informed scheduler that 

Connection established
File Downloaded : NDTiffStack.tif


DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 fl

File Downloaded : NDTiff.index
Files downloaded to: c:\Users\formanj\GitHub\AngelFISH\database\SS004_10min_100nM_7


2025-07-30 13:44:43 |     INFO | [pid 21696] Worker Worker(salt=7273462530, workers=1, host=ens13487, username=formanj, pid=21696) done      AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name=download_data, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json, step_name=download_data, output_path=c:\Users\formanj\GitHub\AngelFISH\database\SS004_10min_100nM_7\default_name\status\step_download_data.txt)
INFO: [pid 21696] Worker Worker(salt=7273462530, workers=1, host=ens13487, username=formanj, pid=21696) done      AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), instance_name=download_data, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json, step_name=download_data, output_path=c:\Users\formanj\GitHub\AngelFISH\da

...

DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_WRITE" (command:0x0009 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_CLOSE" (command:0x0006 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_CREATE" (command:0x0005 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_WRITE" (command:0x0009 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_CLOSE" (command:0x0006 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_CREATE" (command:0x0005 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_WRITE" (command:0x0009 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_WRITE" (command:0x0009 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_WRITE" (command:0x0009 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_WRITE" (command:0x0009 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_WRITE" (comm

Uploaded: dusp1_cellresults.csv
Uploaded: dusp1_clusterresults.csv
Uploaded: dusp1_spotresults.csv
Uploaded: step_detect_dusp1.txt
Uploaded: step_download_data.txt
Uploaded: step_match_cyto_and_nuc.txt
Uploaded: step_measure_cell_props.txt


DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_CREATE" (command:0x0005 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_WRITE" (command:0x0009 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_CLOSE" (command:0x0006 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_CREATE" (command:0x0005 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_WRITE" (command:0x0009 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_CLOSE" (command:0x0006 flags:0x0001)
2025-07-30 13:47:21 |     INFO | Task return_data finished after 12.533s
INFO:sciluigi-interface:Task return_data finished after 12.533s
2025-07-30 13:47:21 |     INFO | [pid 21696] Worker Worker(salt=7273462530, workers=1, host=ens13487, username=formanj, pid=21696) done      AngelFISHLuigiTask(workflow_task=AngelFISHWorkflow(instance_name=sciluigi_workflow, receipt_path=C:\Users\formanj\GitHub\AngelFISH\examples\new_pipeline.json), in

Uploaded: step_segment_cyto.txt
Uploaded: step_segment_nuc.txt


True